# Using Uproot effectively

## Increasingly unnecessary introduction to/motivation for Python

I used to start these tutorials by asking, "Why Python?" but that doesn't seem necessary anymore.

![](img/python-usage.png)

## What is Uproot?

![](img/abstraction-layers.png)

Uproot is an independent implementation of ROOT I/O and only I/O, using standard Python libraries wherever possible.

## Why was it made?

![](img/uproot-awkward-timeline.png)